## Predicts gtown


In [1]:
import gc
import pandas as pd
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfTransformer

pd.set_option('max_colwidth', 120)

In [2]:
vect_name = './output-svc/models/categories_vec_count_nyt_23gram.joblib'
clf_name = './output-svc/models/categories_nyt_model.joblib'
vect = joblib.load(vect_name)
clf = joblib.load(clf_name)
len(vect.vocabulary_)

84180

In [3]:
df = pd.read_csv('/opt/media_text/gtown_us_en_cleaned.csv', nrows=10)
df

,pubDate,language,titl,sourceName,text,url,articleId
0,06 Apr 2010 00:00:00,English,scienceblog traffic hook,Effect Measure Blog,realitybas poll dumb poll second ucla protest ralli send strong messag support scienc anaerob anim discov sea floor ...,http://feedproxy.google.com/~r/scienceblogs/scientificactivist/~3/nfE69Rkya84/scienceblogs_traffic_is_off_th.ph,en_20101122_scienceblogs_363239350009
1,27 Apr 2008 17:01:00,English,shanghai short trip tip hot ticket may day holiday,Loopy News,shanghai short trip tip hot ticket may day holiday sourc shanghai daili author font size bigger middl smaller short ...,http://c.moreover.com/click/here.pl?r139326624,en_20080427_loopynews_139106200059
2,31 Jan 2010 08:14:00,English,edm russiabelaru oil conflict two articl,Loopy News,anngroong edm russiabelaru oil conflict two articl armenian news network groong next articl previou articl main inde...,http://c.moreover.com/click/here.pl?r251134427,en_20100131_loopynews_883099640064
3,15 Jul 2008 13:01:00,English,obama criticis bush iraq polici,Loopy News,barack obama attack presid georg w bush singlemind focu iraq last updatpm bst democrat us presidenti candid barack o...,http://c.moreover.com/click/here.pl?r150662520,en_20080715_loopynews_213228370094
4,05 Jun 2013 00:00:00,English,french report spirit toxin sampl syria,Stars & Stripes,provid,http://www.stripes.com/news/middle-east/french-reporters-spirited-toxin-samples-from-syria-1.224399,en_2013-06-05_a3c8355f1d734bfbce95c12a8686aaa2fd36922a
5,17 Jun 2007 05:00:00,English,dragster spin parad crowd kill,Rootly.com,var fromcnnbeta var betabannerstr click link avail cnn com beta site go back cnn com beta function cnnbetaclos docum...,http://rss.cnn.com/~r/rss/cnn_us/~3/125461448/index.htm,en_20070617_rootly_020191300058
6,14 May 2010 17:07:00,English,phx pd free kidnap drughuman smuggler rival,CurrentEvents.org,welcom tax dollar work rescu drug human smuggler drug human smuggler httpwww azcentr comcommunityp htmlcomment descr...,http://thisbluemarble.com/showthread.php?t=27253&goto=newpos,en_20100514_curevents_052188840003
7,10 Aug 2008 00:00:00,English,understand airport break issu bush airport usher land use regul,Yahoo Alerts,author kingwood observ houston airport system host public hear juli humbl immin plan compat land use regul number lo...,http://www.hcnonline.com/articles/2008/08/10/kingwood_observer/news/ko31airport.tx,en_20080810_yahooalert_239056590006
8,14 Oct 2009 20:23:00,English,nd updateiran progress talk shell repsolag,Loopy News,add inform talk export deal uae entiti london dow jone iran make good progress talk royal dutch shell plc rdsb repso...,http://c.moreover.com/click/here.pl?r226589791,en_20091014_loopynews_708062940049
9,04 Nov 2011 00:00:00,English,liquid capit ’ brian kim plead guilti hedgefund ponzi charg,BB - Bloomberg,brian kim former hedgefund manag liquid capit manag llc accus flee hong kong await trial fraud charg plead guilti ru...,http://www.bloomberg.com/news/2011-11-04/liquid-capital-s-brian-kim-pleads-not-guilty-to-hedge-fund-ponzi-charges.htm,en_20111104_bloomberg6700_908244770022


In [8]:
out_df = pd.DataFrame()
i = 0
for df in pd.read_csv('/opt/media_text/gtown_us_en_cleaned.csv', usecols=['pubDate', 'sourceName', 'text', 'url', 'articleId'], chunksize=10000, iterator=True):
    X = vect.transform(df.text.astype(str))
    tfidf = TfidfTransformer()
    X = tfidf.fit_transform(X)
    y_pred = clf.predict(X)
    df['label_name'] = y_pred
    df.drop('text', axis=1, inplace=True)
    #prob = clf.predict_proba(X)
    prob = clf.decision_function(X)
    prob_df = pd.DataFrame(prob)
    columns = []
    for c in clf.classes_:
        columns.append(c)
    prob_df.columns = columns
    #print prob_df.shape
    df.reset_index(inplace=True, drop=True)
    df = pd.concat([df, prob_df], axis=1)
    #print df.shape
    out_df = out_df.append(df)
    del prob_df
    del df
    del tfidf
    del X
    print(i, out_df.memory_usage(deep=True))
    gc.collect()
    i += 1
    #if i > 2:
    #    break

0 Index                    80
pubDate              770000
sourceName           701210
url                 1318169
articleId            922370
label_name           667712
Arts                  80000
Books                 80000
Business Finance      80000
Classifieds           80000
Dining                80000
Editorial             80000
Foreign News          80000
Health                80000
Leisure               80000
Local                 80000
National              80000
Obits                 80000
Real Estate           80000
Science               80000
Sports                80000
Style                 80000
Travel                80000
dtype: int64
1 Index                160000
pubDate             1540000
sourceName          1401464
url                 2588787
articleId           1844363
label_name          1335894
Arts                 160000
Books                160000
Business Finance     160000
Classifieds          160000
Dining               160000
Editorial            160000
For

13 Index                1120000
pubDate             10780000
sourceName           9812952
url                 17987123
articleId           12909166
label_name           9347468
Arts                 1120000
Books                1120000
Business Finance     1120000
Classifieds          1120000
Dining               1120000
Editorial            1120000
Foreign News         1120000
Health               1120000
Leisure              1120000
Local                1120000
National             1120000
Obits                1120000
Real Estate          1120000
Science              1120000
Sports               1120000
Style                1120000
Travel               1120000
dtype: int64
14 Index                1200000
pubDate             11550000
sourceName          10513290
url                 19294641
articleId           13831395
label_name          10015389
Arts                 1200000
Books                1200000
Business Finance     1200000
Classifieds          1200000
Dining               120

25 Index                2080000
pubDate             20020000
sourceName          18222874
url                 33539198
articleId           23974235
label_name          17358906
Arts                 2080000
Books                2080000
Business Finance     2080000
Classifieds          2080000
Dining               2080000
Editorial            2080000
Foreign News         2080000
Health               2080000
Leisure              2080000
Local                2080000
National             2080000
Obits                2080000
Real Estate          2080000
Science              2080000
Sports               2080000
Style                2080000
Travel               2080000
dtype: int64
26 Index                2160000
pubDate             20790000
sourceName          18922723
url                 34792261
articleId           24896225
label_name          18026801
Arts                 2160000
Books                2160000
Business Finance     2160000
Classifieds          2160000
Dining               216

37 Index                3040000
pubDate             29260000
sourceName          26633354
url                 49260433
articleId           35040062
label_name          25369913
Arts                 3040000
Books                3040000
Business Finance     3040000
Classifieds          3040000
Dining               3040000
Editorial            3040000
Foreign News         3040000
Health               3040000
Leisure              3040000
Local                3040000
National             3040000
Obits                3040000
Real Estate          3040000
Science              3040000
Sports               3040000
Style                3040000
Travel               3040000
dtype: int64
38 Index                3120000
pubDate             30030000
sourceName          27333774
url                 50511149
articleId           35962182
label_name          26038019
Arts                 3120000
Books                3120000
Business Finance     3120000
Classifieds          3120000
Dining               312

49 Index                4000000
pubDate             38500000
sourceName          35044486
url                 64777051
articleId           46105430
label_name          33381573
Arts                 4000000
Books                4000000
Business Finance     4000000
Classifieds          4000000
Dining               4000000
Editorial            4000000
Foreign News         4000000
Health               4000000
Leisure              4000000
Local                4000000
National             4000000
Obits                4000000
Real Estate          4000000
Science              4000000
Sports               4000000
Style                4000000
Travel               4000000
dtype: int64
50 Index                4080000
pubDate             39270000
sourceName          35745095
url                 66026303
articleId           47027766
label_name          34049461
Arts                 4080000
Books                4080000
Business Finance     4080000
Classifieds          4080000
Dining               408

...

445 Index                35680000
pubDate             343420000
sourceName          312607686
url                 576912504
articleId           411251217
label_name          297749084
Arts                 35680000
Books                35680000
Business Finance     35680000
Classifieds          35680000
Dining               35680000
Editorial            35680000
Foreign News         35680000
Health               35680000
Leisure              35680000
Local                35680000
National             35680000
Obits                35680000
Real Estate          35680000
Science              35680000
Sports               35680000
Style                35680000
Travel               35680000
dtype: int64
446 Index                35760000
pubDate             344190000
sourceName          313308246
url                 578247482
articleId           412173150
label_name          298416684
Arts                 35760000
Books                35760000
Business Finance     35760000
Classifieds        

457 Index                36640000
pubDate             352660000
sourceName          321018149
url                 592350156
articleId           422319687
label_name          305761380
Arts                 36640000
Books                36640000
Business Finance     36640000
Classifieds          36640000
Dining               36640000
Editorial            36640000
Foreign News         36640000
Health               36640000
Leisure              36640000
Local                36640000
National             36640000
Obits                36640000
Real Estate          36640000
Science              36640000
Sports               36640000
Style                36640000
Travel               36640000
dtype: int64
458 Index                36720000
pubDate             353430000
sourceName          321718649
url                 593591488
articleId           423242630
label_name          306429225
Arts                 36720000
Books                36720000
Business Finance     36720000
Classifieds        

469 Index                37600000
pubDate             361900000
sourceName          329427901
url                 607834357
articleId           433385415
label_name          313771897
Arts                 37600000
Books                37600000
Business Finance     37600000
Classifieds          37600000
Dining               37600000
Editorial            37600000
Foreign News         37600000
Health               37600000
Leisure              37600000
Local                37600000
National             37600000
Obits                37600000
Real Estate          37600000
Science              37600000
Sports               37600000
Style                37600000
Travel               37600000
dtype: int64
470 Index                37680000
pubDate             362670000
sourceName          330128254
url                 609135463
articleId           434307461
label_name          314440063
Arts                 37680000
Books                37680000
Business Finance     37680000
Classifieds        

481 Index                38560000
pubDate             371140000
sourceName          337838413
url                 623115221
articleId           444451264
label_name          321786562
Arts                 38560000
Books                38560000
Business Finance     38560000
Classifieds          38560000
Dining               38560000
Editorial            38560000
Foreign News         38560000
Health               38560000
Leisure              38560000
Local                38560000
National             38560000
Obits                38560000
Real Estate          38560000
Science              38560000
Sports               38560000
Style                38560000
Travel               38560000
dtype: int64
482 Index                38640000
pubDate             371910000
sourceName          338538284
url                 624399016
articleId           445373063
label_name          322454569
Arts                 38640000
Books                38640000
Business Finance     38640000
Classifieds        

493 Index                39520000
pubDate             380380000
sourceName          346247819
url                 638612853
articleId           455516230
label_name          329800199
Arts                 39520000
Books                39520000
Business Finance     39520000
Classifieds          39520000
Dining               39520000
Editorial            39520000
Foreign News         39520000
Health               39520000
Leisure              39520000
Local                39520000
National             39520000
Obits                39520000
Real Estate          39520000
Science              39520000
Sports               39520000
Style                39520000
Travel               39520000
dtype: int64
494 Index                39600000
pubDate             381150000
sourceName          346948089
url                 639929833
articleId           456437879
label_name          330468007
Arts                 39600000
Books                39600000
Business Finance     39600000
Classifieds        

In [9]:
out_df.shape

(5033525, 22)

In [10]:
out_df.columns = ['date', 'news_source', 'url', 'articleId', 'label_name'] + clf.classes_.tolist()
out_df.to_csv('./output-svc/predicts/gtown_nyt_pred.csv', index=False)